In [1]:
import numpy as np
import json
import os
import re
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3, inception_v3
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [1]:
import numpy as np
import os
import re
import cv2
import Utils as utils
import json
import mediapipe as mp
import math
from csv import writer
from statistics import mean
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
#Guardar datos en un archivo csv
list_data=['03','Smith','Science','1','2','3','4']
with open('CSVFILE.csv', 'a', newline='') as f_object:  
    writer_object = writer(f_object)
    writer_object.writerow(list_data)  
    f_object.close()

In [17]:
#CAMBIO DE IDENTIFICADOR DE FRASES
path_videos = '../../../Videos/VideosLSECv1/'
id_cambio = ['01','03','15','08','04','02','07','19','20','11','18','12','10','14','09','06','17','05','13','16']
print(len(id_cambio))
for folder in os.listdir(path_videos):
    print('-')
    for count, frase in enumerate(os.listdir(path_videos+folder)):
#         print('count: '+str(count))
#         print('frase:' +frase)
        os.rename(path_videos+folder+'/'+frase,path_videos+folder+'/'+'Lsec_'+id_cambio[count])

20
-
-
-


In [37]:
#SACAR IMÁGENES DE UN VIDEO Y OBTENER FPS
path_folders = '../../../Videos/VideosLSECv2/'
datos = {}
total_imgs_fps = []
with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
        ) as holistic:
    for folder in  os.listdir(path_folders):
        datos_folder = {}
        img_total_count = 0
        print(f'{folder}')
        for frase in os.listdir(path_folders+folder):
            datos_frase = {}
            print(f'Frase: {frase}')
            for video in utils.sorted_alphanumeric(os.listdir(path_folders+folder+'/'+frase)):
                datos_video = {}
                img_video_count = 0
                cap = cv2.VideoCapture(path_folders+folder+'/'+frase+'/'+video)
                while(cap.isOpened()):
                    frameId = cap.get(1)
                    ret, frame = cap.read()
                    if (ret != True):
                        break
    #                 cv2.imwrite('../../../Videos/Pruebas/Frase2/'+str(img_video_count)+'.jpg', frame)
                    if(not frame.shape == (360, 480, 3)):
                        frame = cv2.resize(frame, (480,360))
                    image, results = utils.mediapipe_detection(frame, holistic)
                    rh_keypoints_x, rh_keypoints_y, lh_keypoints_x, lh_keypoints_y = utils.extract_keypoints_V3(results)
                    if(rh_keypoints_x and rh_keypoints_y or lh_keypoints_x and lh_keypoints_y):
                        img_video_count += 1
    #             fps = cap.get(cv2.CAP_PROP_FPS)
    #             if(fps < 29.0 or fps > 31.0):
    #                 print(f'{video} FPS: {fps}')
                cap.release()
                datos_video['count'] = img_video_count
                datos_frase[video] = datos_video
                img_total_count += 1
                total_imgs_fps.append(img_video_count)
                #print(video+' count:'+str(datos_video['count']))
            datos_folder[frase] = datos_frase
        datos[folder] = datos_folder
        datos_folder['count'] = img_total_count
        print('count: '+str(datos_folder['count']))

Testing
Frase: Lsec_01
Frase: Lsec_02
Frase: Lsec_03
Frase: Lsec_04
Frase: Lsec_05
Frase: Lsec_06
Frase: Lsec_07
Frase: Lsec_08
Frase: Lsec_09
Frase: Lsec_10
Frase: Lsec_11
Frase: Lsec_12
Frase: Lsec_13
Frase: Lsec_14
Frase: Lsec_15
Frase: Lsec_16
Frase: Lsec_17
Frase: Lsec_18
Frase: Lsec_19
Frase: Lsec_20
count: 1173
Training
Frase: Lsec_01
Frase: Lsec_02
Frase: Lsec_03
Frase: Lsec_04
Frase: Lsec_05
Frase: Lsec_06
Frase: Lsec_07
Frase: Lsec_08
Frase: Lsec_09
Frase: Lsec_10
Frase: Lsec_11
Frase: Lsec_12
Frase: Lsec_13
Frase: Lsec_14
Frase: Lsec_15
Frase: Lsec_16
Frase: Lsec_17
Frase: Lsec_18
Frase: Lsec_19
Frase: Lsec_20
count: 3561
Validating
Frase: Lsec_01
Frase: Lsec_02
Frase: Lsec_03
Frase: Lsec_04
Frase: Lsec_05
Frase: Lsec_06
Frase: Lsec_07
Frase: Lsec_08
Frase: Lsec_09
Frase: Lsec_10
Frase: Lsec_11
Frase: Lsec_12
Frase: Lsec_13
Frase: Lsec_14
Frase: Lsec_15
Frase: Lsec_16
Frase: Lsec_17
Frase: Lsec_18
Frase: Lsec_19
Frase: Lsec_20
count: 1176


In [38]:
json.dump(datos, open('../Logs/DataSetInformation/datasetIMGv3.json', 'w'))

In [22]:
path_folders = '../../../Videos/VideosLSECv1/'
total_imgs = []
total_fps = []
for folder in  os.listdir(path_folders):
    for frase in os.listdir(path_folders+folder):
        for video in utils.sorted_alphanumeric(os.listdir(path_folders+folder+'/'+frase)):
            img_video_count = 0
            cap = cv2.VideoCapture(path_folders+folder+'/'+frase+'/'+video)
            while(cap.isOpened()):
                frameId = cap.get(1)
                ret, frame = cap.read()
                if (ret != True):
                    break
#                 cv2.imwrite('../../../Videos/Pruebas/Frase2/'+str(img_video_count)+'.jpg', frame)
                img_video_count += 1
            fps = cap.get(cv2.CAP_PROP_FPS)
#             if(fps < 29.0 or fps > 31.0):
#                 print(f'{video} FPS: {fps}')
            cap.release()
            total_imgs.append(img_video_count)
            total_fps.append(fps)

In [139]:
print(f'Número total de videos analizados {len(total_imgs_fps)}')
print(f'Promedio de imágenes por video {mean(total_imgs_fps)}')

Número total de videos analizados 1964
Promedio de imágenes por video 110.85183299389001


In [2]:
#DESVIACIÓN ESTANDAR
def desviacion_estandar(datos):
    sumatoria = 0
    promedio_datos = round(mean(datos), 2)
    for dato in datos:
        valor = pow(float(dato)-promedio_datos,2)
        print(f'dato {dato} - {promedio_datos} promedio')
        print(f'Valor: {valor}')
        sumatoria += valor
    print(f'Sumatoria: {sumatoria}')
    resultado = pow(sumatoria/float(len(datos)-1), 0.5)
    return resultado

In [4]:
datos = [78.8,65.0,73.1,50.0,88.2,76.5,73.3,75.0,55.6,94.7,75.0,53.3,85.7,88.2,76.5,64.7,82.4,65.4,57.1,46.2]
len(datos)

20

In [5]:
print(desviacion_estandar(datos))

dato 75.0 - 72.13 promedio
Valor: 8.236900000000027
dato 86.7 - 72.13 promedio
Valor: 212.28490000000022
dato 73.1 - 72.13 promedio
Valor: 0.9408999999999978
dato 50.0 - 72.13 promedio
Valor: 489.7368999999998
dato 88.2 - 72.13 promedio
Valor: 258.24490000000026
dato 76.5 - 72.13 promedio
Valor: 19.09690000000004
dato 73.3 - 72.13 promedio
Valor: 1.368900000000004
dato 75.0 - 72.13 promedio
Valor: 8.236900000000027
dato 55.6 - 72.13 promedio
Valor: 273.2408999999998
dato 94.7 - 72.13 promedio
Valor: 509.40490000000034
dato 75.0 - 72.13 promedio
Valor: 8.236900000000027
dato 53.3 - 72.13 promedio
Valor: 354.5688999999999
dato 85.7 - 72.13 promedio
Valor: 184.1449000000002
dato 88.2 - 72.13 promedio
Valor: 258.24490000000026
dato 76.5 - 72.13 promedio
Valor: 19.09690000000004
dato 64.7 - 72.13 promedio
Valor: 55.20489999999989
dato 82.4 - 72.13 promedio
Valor: 105.47290000000021
dato 65.4 - 72.13 promedio
Valor: 45.29289999999986
dato 57.1 - 72.13 promedio
Valor: 225.9008999999998
dato 4

In [99]:
#OBTENER IMÁGENES EN BASE AL NÚMERO LÍMITE Y SU NÚMERO DE FRAMES TOTALES
def obtener_imagenes(lista_imagenes):
    print(f'Lista de valores: {lista_imagenes}')
    MAX_FRAMES = 100
    pasos = len(lista_imagenes)/MAX_FRAMES
    if(pasos < 1):
        pasos = 1
    imagenes_salida = []
    id_imagen = 0
    for i, imagen  in enumerate(lista_imagenes):
#         print(id_imagen)
        if(id_imagen == 0 and len(imagenes_salida) < MAX_FRAMES):
            imagenes_salida.append(imagen)
            id_imagen += pasos
#             print(imagen)
        elif(int(math.floor(id_imagen)) == i and len(imagenes_salida) < MAX_FRAMES):
            imagenes_salida.append(imagen)
            id_imagen += pasos
#             print(imagen)
        
    return imagenes_salida

In [100]:
#     x1 = [x for x in range(100)]
#     x2 = [x for x in range(163)]
#     x3 = [x for x in range(234)]
imgs = obtener_imagenes(lista_imagenes = [x for x in range(88)])

Lista de valores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]


In [3]:
#OBTENER LISTA DE IMAGENES POR VIDEO MEDIANTE DICCIONARIO
def obtener_valores(diccionario):
    lista_imgs_video = []
    for keys, values in diccionario.items():
        lista_imgs = []
        for key, value in values.items():
            lista_imgs.append(value['count'])
        lista_imgs_video.append(lista_imgs)
    return lista_imgs_video

In [33]:
#OBTENER LISTA DE IMAGENES POR VIDEO MEDIANTE DICCIONARIO v2
def obtener_valores_v2(diccionario):
    lista_imgs_video = []
    for x, y in diccionario.items():
        for keys, values in y.items():
            for key, value in values.items():
                lista_imgs_video.append(value['count'])
    return lista_imgs_video

In [4]:
datos_videos = json.load(open('../Logs/DataSetInformation/datasetIMGv2.json'))
del datos_videos['Validating']['count']

In [5]:
del datos_videos['Testing']['count']

In [32]:
lista_imgs_video = obtener_valores_v2(datos_videos)

{'Lsec_01': {'1.mp4': {'count': 89}, '2.mp4': {'count': 102}, '3.mp4': {'count': 80}, '4.mp4': {'count': 134}, '5.mp4': {'count': 138}, '6.mp4': {'count': 68}, '7.mp4': {'count': 70}, '8.mp4': {'count': 83}, '9.mp4': {'count': 45}, '10.mp4': {'count': 127}, '11.mp4': {'count': 145}, '12.mp4': {'count': 90}, '13.mp4': {'count': 72}, '14.mp4': {'count': 64}, '15.mp4': {'count': 76}, '16.mp4': {'count': 126}, '17.mp4': {'count': 64}, '18.mp4': {'count': 101}, '19.mp4': {'count': 54}}, 'Lsec_02': {'1.mp4': {'count': 106}, '2.mp4': {'count': 75}, '3.mp4': {'count': 95}, '4.mp4': {'count': 98}, '5.mp4': {'count': 99}, '6.mp4': {'count': 40}, '7.mp4': {'count': 92}, '8.mp4': {'count': 50}, '9.mp4': {'count': 58}, '10.mp4': {'count': 83}, '11.mp4': {'count': 54}, '12.mp4': {'count': 111}, '13.mp4': {'count': 115}, '14.mp4': {'count': 115}, '15.mp4': {'count': 67}, '16.mp4': {'count': 68}, '17.mp4': {'count': 102}, '18.mp4': {'count': 97}, '19.mp4': {'count': 62}, '20.mp4': {'count': 73}}, 'Lse

In [34]:
len(lista_imgs_video)

1970

In [25]:
IMG_SIZE = 50

In [47]:
#Ver el resultado de aplicar VGG16
def build_feature_extractor():
    feature_extractor = VGG16(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 6),
    )
    preprocess_input = inception_v3.preprocess_input

    inputs = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()

ValueError: The input must have 3 channels; got `input_shape=(50, 50, 6)`

In [31]:
def esPar(numero):
    if(numero % 2 == 0):
        return True
    else:
        return False

In [8]:
datos_val = np.load('../DataSet/v1/Validating/x_val.npy')
datos_val_1 = datos_val[1]

In [15]:
datos_val_1.shape

(128, 50, 50, 3)

In [16]:
vc = feature_extractor.predict(datos_val_1)

In [38]:
vc[0]

array([2.89036632e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.26217031e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.36034703e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.38652825e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.51266500e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.91423076e-01, 0.00000000e+00,
       4.01559472e-03, 3.65713120e-01, 2.32274145e-01, 0.00000000e+00,
       0.00000000e+00, 1.11285999e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.82458377e-01,
      

In [43]:
nuevos_datos = []
for i, dato in enumerate(datos_val_1): 
    if(esPar(i)):
        temp = dato
    else:
        nuevos_datos.append(np.concatenate([temp,dato], axis=2))

In [45]:
np.array(nuevos_datos)

array([[[[0.27058824, 0.45882353, 0.16470588, 0.        , 0.        ,
          0.        ],
         [0.27058824, 0.45490196, 0.16862745, 0.        , 0.        ,
          0.        ],
         [0.28627451, 0.47058824, 0.18431373, 0.        , 0.        ,
          0.        ],
         ...,
         [0.27843137, 0.44313725, 0.18431373, 0.        , 0.        ,
          0.        ],
         [0.29019608, 0.44705882, 0.2       , 0.        , 0.        ,
          0.        ],
         [0.29411765, 0.44705882, 0.20784314, 0.        , 0.        ,
          0.        ]],

        [[0.2627451 , 0.44705882, 0.15686275, 0.        , 0.        ,
          0.        ],
         [0.26666667, 0.45098039, 0.16078431, 0.        , 0.        ,
          0.        ],
         [0.27843137, 0.4627451 , 0.17254902, 0.        , 0.        ,
          0.        ],
         ...,
         [0.27843137, 0.43921569, 0.18431373, 0.        , 0.        ,
          0.        ],
         [0.28627451, 0.44313725, 0.1960

In [46]:
vc2 = feature_extractor.predict(nuevos_datos)

ValueError: in user code:

    C:\Users\juan.guevara01\AppData\Roaming\Python\Python36\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\juan.guevara01\AppData\Roaming\Python\Python36\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\juan.guevara01\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\juan.guevara01\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\juan.guevara01\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C

In [32]:
a1 = np.random.rand(50,50,3)
a2 = np.zeros((50,50,3))

In [35]:
a3 = np.concatenate([a1,a2], axis=2)

In [48]:
a1[0]

array([[0.06793716, 0.92054129, 0.84316383],
       [0.91509683, 0.01038324, 0.33688999],
       [0.7047952 , 0.53487492, 0.31990465],
       [0.85399544, 0.91827095, 0.18723671],
       [0.48140133, 0.57572569, 0.68038966],
       [0.6732007 , 0.4981858 , 0.65528553],
       [0.4920695 , 0.4315164 , 0.23412359],
       [0.26580579, 0.6122153 , 0.55497939],
       [0.37675043, 0.17070257, 0.51429213],
       [0.60360422, 0.93008668, 0.2188045 ],
       [0.19698069, 0.51286143, 0.6016876 ],
       [0.81143789, 0.07941981, 0.89482017],
       [0.07225889, 0.43078221, 0.36386365],
       [0.67131773, 0.49244218, 0.78633965],
       [0.54457383, 0.46186743, 0.25056437],
       [0.00471691, 0.28616905, 0.6250067 ],
       [0.4421959 , 0.62647534, 0.89343874],
       [0.2413369 , 0.9805093 , 0.0598101 ],
       [0.07765005, 0.01525082, 0.8120835 ],
       [0.68724723, 0.88320805, 0.86383764],
       [0.25187939, 0.90080301, 0.59389093],
       [0.37230874, 0.25886922, 0.21736384],
       [0.

In [3]:
with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
        ) as holistic:
    img_video_count = 0
    cap = cv2.VideoCapture('../../../Videos/Pruebas/Lsec_01/2.mp4')
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret, frame = cap.read()
        if (ret != True):
            break
        if(not frame.shape == (360, 480, 3)):
            frame = cv2.resize(frame, (480,360))
        image, results = utils.mediapipe_detection(frame, holistic)
        rh_keypoints_x, rh_keypoints_y, lh_keypoints_x, lh_keypoints_y = utils.extract_keypoints_V3(results)
        if(rh_keypoints_x and rh_keypoints_y or lh_keypoints_x and lh_keypoints_y):
            img_video_count += 1
            cv2.imwrite('../../../Videos/Pruebas/2/'+str(img_video_count)+'.jpg', frame)
#             fps = cap.get(cv2.CAP_PROP_FPS)
#             if(fps < 29.0 or fps > 31.0):
#                 print(f'{video} FPS: {fps}')
    cap.release()
print(img_video_count)

KeyboardInterrupt: 

In [6]:
json.load(open('../Logs/DataSetInformation/datasetIMGv2.json'))

{'Testing': {'Lsec_01': {'1.mp4': {'count': 89},
   '2.mp4': {'count': 102},
   '3.mp4': {'count': 80},
   '4.mp4': {'count': 134},
   '5.mp4': {'count': 138},
   '6.mp4': {'count': 68},
   '7.mp4': {'count': 70},
   '8.mp4': {'count': 83},
   '9.mp4': {'count': 45},
   '10.mp4': {'count': 127},
   '11.mp4': {'count': 145},
   '12.mp4': {'count': 90},
   '13.mp4': {'count': 72},
   '14.mp4': {'count': 64},
   '15.mp4': {'count': 76},
   '16.mp4': {'count': 126},
   '17.mp4': {'count': 64},
   '18.mp4': {'count': 101},
   '19.mp4': {'count': 54}},
  'Lsec_02': {'1.mp4': {'count': 106},
   '2.mp4': {'count': 75},
   '3.mp4': {'count': 95},
   '4.mp4': {'count': 98},
   '5.mp4': {'count': 99},
   '6.mp4': {'count': 40},
   '7.mp4': {'count': 92},
   '8.mp4': {'count': 50},
   '9.mp4': {'count': 58},
   '10.mp4': {'count': 83},
   '11.mp4': {'count': 54},
   '12.mp4': {'count': 111},
   '13.mp4': {'count': 115},
   '14.mp4': {'count': 115},
   '15.mp4': {'count': 67},
   '16.mp4': {'count'

In [9]:
for i in range(3):
    os.system('cmd /k "date"') 

In [13]:
list_images = os.listdir('../../../Videos/Pruebas/Lsec_01')

In [14]:
len(list_images)

3

In [16]:
counter = len(list_images)+1
for video in os.listdir('../../../Videos/Pruebas/VideosAumentados'):
    os.rename('../../../Videos/Pruebas/VideosAumentados/'+video, '../../../Videos/Pruebas/Lsec_01/'+str(counter)+'.mp4')
    counter += 1

In [34]:
#AUMENTAR VIDEOS POR CADA FRASE
path_inicio = '../../../Videos/VideosLSECv1/'
path_destino = '../../../Videos/Pruebas/VideosAumentados/'
max_clips = '2'
for directory in os.listdir(path_inicio):
#     if(directory == 'Testing'):
#         continue
    print(directory)
    for frase in os.listdir(path_inicio+directory):
#         if(directory == 'Training' and int(frase.split('_')[1])<7):
#             continue
        path_origen = path_inicio+directory+'/'+frase
        #os.system('python video_augmentation_code.py --main-folder '+path_origen+' --output-folder '+path_destino+' --max-clips '+max_clips)
        counter = len(os.listdir(path_origen))+1
        print(frase, counter)
#         for video in os.listdir(path_destino):
#             os.rename(path_destino+video, path_origen+'/'+str(counter)+'.mp4')
#             counter += 1

Training
Lsec_07 62
Lsec_08 61
Lsec_09 62
Lsec_10 61
Lsec_11 61
Lsec_12 58
Lsec_13 58
Lsec_14 61
Lsec_15 61
Lsec_16 60
Lsec_17 59
Lsec_18 58
Lsec_19 63
Lsec_20 61
Validating
Lsec_01 21
Lsec_02 21
Lsec_03 21
Lsec_04 21
Lsec_05 19
Lsec_06 21
Lsec_07 20
Lsec_08 21
Lsec_09 21
Lsec_10 21
Lsec_11 21
Lsec_12 20
Lsec_13 20
Lsec_14 21
Lsec_15 21
Lsec_16 20
Lsec_17 20
Lsec_18 20
Lsec_19 21
Lsec_20 21


In [28]:
counter = 61
for video in os.listdir(path_destino):
    os.rename(path_destino+video, '../../../Videos/VideosLSECv1/Training/Lsec_06/'+str(counter)+'.mp4')
    counter+=1

In [33]:
int(frase.split('_')[1])

84